# Graph Visualizations

Need to clean this up. But basically this is a visualization of who tags who and shows off any communities (groups that tag eachother a lot) in a nice graph

TODO: Display how many times each account tweeted an @ 

In [1]:
len(tweets[tweets.extended_tweet.str.contains("JustinT")])

NameError: name 'tweets' is not defined

In [36]:
import pandas as pd
import urllib.request
import ast
import pandas as pd 

target_url="https://swift-yeg.cloud.cybera.ca:8080/v1/AUTH_233e84cd313945c992b4b585f7b9125d/geeky-summit/tweets_cleaned.csv"
file_name="tweets_cleaned.csv"
urllib.request.urlretrieve(target_url, file_name)

tweets = pd.read_csv(file_name,parse_dates=['created_at_date']) 
tweets.head()

,created_at_date,hashtags_string,user_string,user_location,lang,longitude,latitude,name,screen_name,extended_tweet,extended_tweet_cleaned
0,2018-11-02 21:01:56,,Symin16,Toronto ✈ Calgary,en,NaN,NaN,♠,jessmayumba85,@Symin16 I’d like to know who,I’d like to know who
1,2018-11-02 21:02:01,,TwoCanSamAdams,YYC,en,NaN,NaN,hannahrae cuddleslug,thimblewad,@TwoCanSamAdams Legit. There are still chunks ...,Legit. There are still chunks in the sink an...
2,2018-11-02 21:02:05,job Calgary SupplyChain Hiring CareerArc,,Calgary,en,51.004583,-114.007914,TMJ - CAL Manuf Jobs,tmj_cal_manuf,Can you recommend anyone for this #job in #Cal...,"Can you recommend anyone for this in , AB? ..."
3,2018-11-02 21:02:10,,,🌎📱,en,NaN,NaN,Sunny Rai,TheSunsRay,Kids See Ghosts: love this track,Kids See Ghosts: love this track
4,2018-11-02 21:02:13,Calgary job,,Calgary,en,50.997882,-114.074005,TMJ-CAL Retail Jobs,tmj_cal_retail,"See our latest #Calgary, AB #job and click to ...","See our latest , AB and click to apply: bar..."


In [33]:
import networkx as nx
import matplotlib.pyplot as plt

def gen_network(row):
    user = row.screen_name
    tagged = row.user_string
    if tagged != ' ':
        return set(tagged.split())
    else:
        return None

In [34]:
tweets['tagdict']= tweets.apply(gen_network, axis=1)

In [39]:
def remove_empty(row):
    # print(row.user_string, "SDFSDF")
    return row.user_string.split()
    
tweet_dict = tweets.groupby(by='screen_name')['user_string'].apply(list).to_dict()
for key in tweet_dict:
    new = []
    for tags in tweet_dict[key]: 
        new.append(tags.split())
    tweet_dict[key] =  [item for sublist in new for item in sublist]
    
tweet_dict

{'1001cjd': [],
 '1015KooLFM': ['KristenandTay', 'robandjoelle'],
 '107Jackson': [],
 '11_Jamieson': [],
 '16guitars': [],
 '1durful': [],
 '2digithandicap': ['ezralevant'],
 '2oolman': [],
 '2silvercats': ['NordyJenn',
  'ABOotlanders',
  'MrStevenCree',
  'krisboydcharity',
  'MrStevenCree'],
 '2winmomCurtis': [],
 '311calgary': [],
 '3hreefaced': [],
 '3thoalgarneen': [],
 '420Cannaisseur': [],
 '4connection_': ['eventbrite'],
 '4mcadam': ['VZWSupport', 'mcjesseb'],
 '4ulocal_allen': [],
 '4worklifeenergy': ['FlyYYC', 'YYC_Taxi'],
 '5_Vines': [],
 '67rikkus': [],
 '959chfm': ['959chfm', 'SandalsResorts', 'SandalsResorts'],
 '9allie9': [],
 'ABCraftCouncil': ['cspaceprojects', 'studiocprospect'],
 'AB_PETE': ['Tyler_Pelke'],
 'AB_SpeedSkating': ['ISU_Speed', 'CBCOlympics', 'jrcelski', 'SkatingSource'],
 'ACCLabelinc': [],
 'ADBradac': ['PechaKuchaYYC'],
 'AKA_KEVIN_ALEX': ['Inc', 'Inc', 'telfordk', 'VeteransENG_CA', 'astonmartin'],
 'ALLSAINTSccsd': [],
 'AMBruzga': [],
 'AMSimoneau'

In [40]:
tags = list(tweets[tweets.tagdict.notnull()].tagdict)
result = {}
for d in tags:
    result.update(d)
g= nx.from_dict_of_lists(tweet_dict)

g

AttributeError: 'DataFrame' object has no attribute 'tagdict'

# Most connected subgraph (largest community) 

Clean this up or get rid of it probably? 

In [ ]:
a=max(nx.connected_component_subgraphs(g), key=len)
plt.figure(figsize=(10,10))
nx.draw(a, node_size=10, label_size=5, with_labels=True)
plt.draw()
plt.show()

# Who is the most 'connected' node (most tagged account) 
It's the library - talk about why there's only a few (as compared to so many in the exploration stage) 

TODO: Wrap this in a widget where the users can explore the most connected tags going down from most to least popular 

In [ ]:
from operator import itemgetter
plt.figure(figsize=(10,10))
node_and_degree = g.degree()
(largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]
# Create ego graph of main hub
hub_ego = nx.ego_graph(g, largest_hub)
# Draw graph
pos = nx.spring_layout(hub_ego)
nx.draw(hub_ego, pos, node_color='b', node_size=50, with_labels=True, edge_color = 'b')
# Draw ego as large and red
nx.draw_networkx_nodes(hub_ego, pos, nodelist=[largest_hub], node_size=300, node_color='r', with_labels=True)
plt.show()

In [7]:
node_and_degree['realDonaldTrump'] 

NameError: name 'node_and_degree' is not defined

# This will be a widget so people can explore most connected stuff

In [ ]:
hub_ego = nx.ego_graph(g, 'realDonaldTrump')
pos = nx.spring_layout(hub_ego)
nx.draw(hub_ego, pos, node_color='b', node_size=50, with_labels=True, edge_color = 'b')
# Draw ego as large and red
nx.draw_networkx_nodes(hub_ego,
                       pos, 
                       nodelist=['realDonaldTrump'], 
                       node_size=300, 
                       node_color='r', 
                       with_labels=True)
plt.show()


In [ ]:
len(tweets[tweets.extended_tweet.str.contains('realDonaldTrump')]['screen_name'].unique())